## Libraries

In [1]:
from Bio import SeqIO
import re
import os

## Variables

In [77]:
project = 'Suthaus_2022'
cell = 'cellCombined'
marker = 'Full18S'
sim = 'sim90'
denoise_method = 'RAD'
raw_data = os.path.join('..', 'raw_data')
alignment_dir = os.path.join(raw_data, 'reference_alignments', 'pr2')
alignment_file = 'pr2_version_4.14.0_SSU_UTAX.fasta'
otu_eukaryotes_dir = os.path.join(raw_data, 'OTU_eukaryotes', project, marker, cell, sim, denoise_method)
tax_assign_dir = os.path.join(raw_data, 'tax_assign_results', project, marker, cell, sim, denoise_method)
otu_nonchimeric_dir = os.path.join(raw_data, 'OTU_nonchimeric', project, marker, cell, sim, denoise_method)

# Subsetting PR2 alignment

Subseting the PR2 alignment to reduce computational load. Keeping a maximum of limited occurrences per taxonomic level (e.g. max 3 occurrences per genus) to retain diversity while significantly reducing the size of the alignment.

In [3]:
# Load your sequences
input_file = os.path.join(alignment_dir, alignment_file)
output_file = os.path.join(alignment_dir, 'pr2_version_4.14.0_SSU_UTAX_subset.fasta')


# Set parameters
limit = 1 # threshold: max sequences for one taxonomic level (e.g. genus or family)
tax_level = 'f' # taxonomic level (e.g., f for family, g for genus, and so on)
max_length = 3000  # set maximum sequence length


# Initialize dictionary to hold sequences per taxon level
tax_dict = {}

# Read and parse sequences
with open(input_file, 'r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        # Check the length of the sequence
        if len(record.seq) <= max_length:
            # Extract genus information from header
            tax_match = re.search(rf'{tax_level}:([^,]+)', record.description)
            if tax_match:
                taxon = tax_match.group(1)

                # Add sequence to tax_dict, if taxon count is below the threshold/limit
                if taxon not in tax_dict:
                    tax_dict[taxon] = [record]
                elif len(tax_dict[taxon]) < limit:
                    tax_dict[taxon].append(record)

# Write filtered sequences to a new file
with open(output_file, 'w') as output_handle:
    for taxon_records in tax_dict.values():
        SeqIO.write(taxon_records, output_handle, "fasta")

In [4]:
!grep '>' $output_file | wc -l

2162


In [25]:
seqs_to_remove = ['X93221.1.842_U;tax=k:Eukaryota,d:Excavata,p:Discoba,c:Heterolobosea,o:Tetramitia_VI,f:Vahlkampfiidae_VI,g:Willaertia_VI,s:Willaertia_VI_magna', 
                  'JN082914.1.1718_U;tax=k:Eukaryota,d:Opisthokonta,p:Metazoa,c:Arthropoda,o:Hexapoda,f:Hexapoda_X,g:Tridactylophagus,s:Tridactylophagus_sp.',
                  'JN566139.1.2860_U;tax=k:Eukaryota,d:Excavata,p:Discoba,c:Euglenida,o:Euglenida_X,f:Euglenida_XX,g:Heteronema,s:Heteronema_scaphurum',
                  'AJ937879.1.1048_U;tax=k:Eukaryota,d:Opisthokonta,p:Metazoa,c:Myxozoa,o:Myxozoa_X,f:Myxozoa_XX,g:Myxozoa_XXX,s:Myxozoa_XXX_sp.',
                  'JQ519509.1.1706_U;tax=k:Eukaryota,d:Amoebozoa,p:Lobosa,c:Tubulinea,o:Arcellinida,f:Difflugiidae,g:Netzelia,s:Netzelia_wailesi',
                  'DQ205365.1.1268_U;tax=k:Eukaryota,d:Rhizaria,p:Foraminifera,c:Globothalamea,o:Rotaliida,f:Cibicididae,g:Cibicidoides,s:Cibicidoides_refulgens',
                  'FJ705910.1.1268_U;tax=k:Eukaryota,d:Rhizaria,p:Foraminifera,c:Globothalamea,o:Rotaliida,f:Fursenkoinacea,g:Virgulina,s:Virgulina_concava',
                  'DQ205383.1.1263_U;tax=k:Eukaryota,d:Rhizaria,p:Foraminifera,c:Globothalamea,o:Rotaliida,f:Pulleniidae,g:Pullenia,s:Pullenia_subcarinata',
                  'DQ205389.1.1342_U;tax=k:Eukaryota,d:Rhizaria,p:Foraminifera,c:Globothalamea,o:Rotaliida,f:Buliminidae,g:Bulimina,s:Bulimina_marginata',
                  'FJ705907.1.1336_U;tax=k:Eukaryota,d:Rhizaria,p:Foraminifera,c:Globothalamea,o:Rotaliida,f:Cassidulinidae,g:Cassidulina,s:Cassidulina_laevigata',
                  'DQ205362.1.1195_U;tax=k:Eukaryota,d:Rhizaria,p:Foraminifera,c:Globothalamea,o:Rotaliida,f:Planorbulinidae,g:Hyalinea,s:Hyalinea_balthica',
                  'AJ628838.1.1178_U;tax=k:Eukaryota:plas,d:Alveolata:plas,p:Dinoflagellata:plas,c:Dinophyceae:plas,o:Gonyaulacales:plas,f:Ceratiaceae:plas,g:Neoceratium:plas,s:Neoceratium_horridum:plas',
                  'DQ264858.1.1219_U;tax=k:Eukaryota:plas,d:Alveolata:plas,p:Dinoflagellata:plas,c:Dinophyceae:plas,o:Gonyaulacales:plas,f:Gonyaulacaceae:plas,g:Lingulodinium:plas,s:Lingulodinium_polyedra:plas',
                  'LN735525.1.1205_U;tax=k:Eukaryota:plas,d:Alveolata:plas,p:Dinoflagellata:plas,c:Dinophyceae:plas,o:Suessiales:plas,f:Symbiodiniaceae:plas,g:Breviolum:plas,s:Breviolum_sp.:plas',
                  'AY123745.1.924_UC;tax=k:Eukaryota,d:Opisthokonta,p:Fungi,c:Ascomycota,o:Pezizomycotina,f:Sordariomycetes,g:Sordariomycetes_X,s:Sordariomycetes_X_sp.']

substring_to_remove = ['k:Bacteria', 'k:Archaea']

In [26]:
# Load the sequences into a list
input_file =os.path.join(alignment_dir, 'pr2_version_4.14.0_SSU_UTAX_subset.fasta')

sequences = list(SeqIO.parse(input_file, "fasta"))

In [28]:
# Filtering based on seqs_to_remove:
filtered_sequences = [seq for seq in sequences if seq.id not in seqs_to_remove]
# Filtering based on substrings_to_remove:
filtered_sequences = [seq for seq in filtered_sequences if not any(sub in seq.id for sub in substring_to_remove)]

In [33]:
output_file = os.path.join(alignment_dir, 'pr2_version_4.14.0_SSU_UTAX_subset_filtered.fasta')

with open(output_file, "w") as output_handle:
    SeqIO.write(filtered_sequences, output_handle, "fasta")

In [29]:
len(filtered_sequences)

1600

# Extracting the sequence names and taxonomic paths

In [34]:
input_file = os.path.join(alignment_dir, 'pr2_version_4.14.0_SSU_UTAX_subset_filtered.fasta')
output_file = os.path.join(alignment_dir, 'ids_list.txt')

# Open the input file for reading and output file for writing
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    # Read through each line in the input file
    for line in infile:
        # If the line starts with '>', it's a header line
        if line.startswith('>'):
            # Write the line to the output file, excluding the '>' and trimming whitespace
            outfile.write(line[1:].strip() + '\n')  # write the id without '>' and add a newline


In [ ]:
# Creating taxon file

input_file = os.path.join(alignment_dir, 'ids_list.txt')
output_file = os.path.join(alignment_dir, 'taxon_file.txt')

# Open the input file for reading and output file for writing
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    # Read through each line in the input file
    for line in infile:
        # If the line starts with '>', it's a header line
        if line.startswith('>'):
            # Write the line to the output file, excluding the '>' and trimming whitespace
            outfile.write(line[1:].strip() + '\n')  # write the id without '>' and add a newline

AJ875130.1.1731_U;tax=k:Eukaryota,d:Opisthokonta,p:Metazoa,c:Nematoda,o:Enoplea,f:Enoplea_X,g:Enoplea_XX,s:Enoplea_XX_sp.

# Removing taxopath from the PR2 subset alignment

In [ ]:
from Bio import SeqIO

input_file = os.path.join(alignment_dir, 'pr2_version_4.14.0_SSU_UTAX_subset.fasta')
output_file = os.path.join(alignment_dir, 'pr2_version_4.14.0_SSU_UTAX_subset_modified.fasta')

# Open the input file, parse sequences, modify IDs, and write to output file
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    # Iterate over each record in the input file
    for record in SeqIO.parse(infile, "fasta"):
        # Split the ID at the '.' character and keep only the first part
        new_id = record.id.split('.')[0]
        # Update the record's ID with the new ID
        record.id = new_id
        record.description = new_id  # Update the description to match the new ID
        # Write the modified record to the output file
        SeqIO.write(record, outfile, "fasta")

# Using Gblocks

In [ ]:
GBLOCKS = '../raw_data/packages/Gblocks_0.91b/Gblocks'
ALIGNMENT = '../raw_data/reference_alignments/pr2/pr2_version_4.14.0_SSU_UTAX_subset_modified_align.fasta'

!$GBLOCKS $ALIGNMENT -t=d -b4=2 -b5=H -b3=2000 -e=.gb1

# Subset Query Sequences

## Filter out all the bacterial and archaeal sequences from the OTUs

In [88]:
# Functions

# Extract sequence IDs with Bacteria or Archaea in their taxonomic assignment
def find_bacteria_archaea(tax_assign_dir, file):
    '''
    Extract sequence IDs with Bacteria or Archaea in their taxonomic assignment.

    Parameters:
    - tax_assign_dir (str): Directory path containing taxonomic assignment files.
    - file (str): Taxonomic assignment file.

    Returns:
    - bacterial_and_archaeal_ids (set): Set of sequence IDs with either Bacteria or Archaea in their taxonomic assignment.
    '''
    bacterial_and_archaeal_ids = []
    with open(os.path.join(tax_assign_dir, file), "r") as f:
        for line in f:
            parts = line.strip().split("\t")
            taxonomy = parts[1]
            seq_id = parts[0]
            if 'k:Bacteria' in taxonomy or 'k:Archaea' in taxonomy:
                bacterial_and_archaeal_ids.append(seq_id)
    return bacterial_and_archaeal_ids

# Filter fasta files
def filter_fasta_file(input_file, output_file, ids_to_exclude):
    '''
    Filters sequences in a fasta file based on a list of sequence IDs to exclude.

    Parameters:
    - input_file (str): Path to the input fasta file.
    - output_file (str): Path where the filtered fasta file should be saved.
    - ids_to_exclude (set): Set of sequence IDs to exclude.
    '''
    sequences = [record for record in SeqIO.parse(input_file, "fasta") if record.id not in ids_to_exclude]
    SeqIO.write(sequences, output_file, "fasta")

In [81]:
# list of samples you want to filter
samples = ['A3', 
           'NH1', 
           'NH4', 
           'Sim17', 
           'Sim22', 
           'Th16', 
           'Th38', 
           'Th40', 
           'X17007']

In [89]:
# Usage

# Loop through the samples
for sample in samples:
    
    # Step 1: Extract sequence IDs with Bacteria or Archaea from the taxonomy assignment file
    file_name = f'blast6_{sample}_18S.tab'
    bacterial_and_archaeal_ids = find_bacteria_archaea(tax_assign_dir, file_name)
    
    # Define input and output paths for fasta files
    input_fasta_file = os.path.join(otu_nonchimeric_dir, f'{sample}_18S_otu.fasta')
    output_fasta_file = os.path.join(otu_eukaryotes_dir, f'{sample}_18S_otu_filtered.fasta')
    
    
    # Step 2: Filter fasta files and save them to the designated directory
    filter_fasta_file(input_fasta_file, output_fasta_file, bacterial_and_archaeal_ids)